# Take Control of GPT : Use the API. Don't let the API use you

### My first openai API Project!!!

Here, I use the openai Python lib and some custom classes to handle processing prompts and responses. I store the results in a pandas dataframe, then format the code in a pandas 'style' object. I can use the dataframe and other objects (like the multiline string of prompts) to manage context for later prompts. Yes, context from prompts and answers. So much more to do with this. But this is a start.

## Setup

In [1]:
%%html
<style>
    /* Enable word wrap */
    .rendered_html table, .rendered_html th, .rendered_html tr, .rendered_html td {
      white-space: nowrap;
      overflow: hidden;
      text-overflow: ellipsis;
    }
</style>

### Imports

In [2]:
# ...
import openai
import os
import pprint
import sys
from pprint import pprint
import json
import pyarrow as pa
import pandas as pd

lib = os.path.abspath(os.path.join('..', '..', 'libs'))
sys.path.append(lib)

from MessageParser import MessageParser
from ResponseParser import ResponseParser
from PromptLogs import PromptLogs


### Config

In [3]:
# ...
# *******************************************************************************************************
# SET THE API KEY
# ================
# -- set the env variable OPENAI_API_KEY to your OpenAI API key       <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
openai.api_key = os.getenv("OPENAI_API_KEY")
# *******************************************************************************************************

# Setup up pandas properties
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent line break in DataFrame
pd.set_option('display.max_colwidth', None)  # Display the full content of cells



## Main code

Prep the system prompts

In [4]:
sys_prompt_list = []
sys_prompt_1 = """You are a software expert."""
sys_prompt_2 = """Use. "```python" style for code blocks."""
sys_prompt_3 = """Include the name for a file where I would store individual code for functions or classes."""

sys_prompt_list.append(sys_prompt_1)
sys_prompt_list.append(sys_prompt_2)
sys_prompt_list.append(sys_prompt_3)

sys_prompt_string = "".join(sys_prompt_list)

### a PromptLogs object will hold all the individual individual parser objects, which hold the response from a single exchange

In [5]:
logs = PromptLogs()

Generate the message and make the call

In [6]:
message_list = f"""
system: {sys_prompt_string}
user: Write python code for fibonacci sequence. also write a function to zero pad a string to 10 chars.
"""

# a MessageParser object will process the message_list and make an acceptable messages object
message_parser = MessageParser(message_list)
messages = message_parser.parse()

# Make the openai call and get the response
# todo: I'll abstract this away in the future
response = openai.ChatCompletion.create(model='gpt-3.5-turbo', temperature=1.3, max_tokens=300, messages = messages)


A response parser will process the response

In [7]:
response_parser = ResponseParser(response=response)

See a pandas dataframe of the logged response

In [8]:
response_parser.df

,id,created,model,usage,all_code,code_blocks,content
0,chatcmpl-756VFQYJPiS3pWVPJZ6oVdGWjlolU,1.681451e+09,gpt-3.5-turbo-0301,66.0,"def&nbsp;fibonacci(n):<br>&nbsp;&nbsp;if&nbsp;n&nbsp;<&nbsp;0:<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;""Invalid&nbsp;input!!!&nbsp;Input&nbsp;should&nbsp;be&nbsp;a&nbsp;positive&nbsp;integer.""<br>&nbsp;&nbsp;elif&nbsp;n&nbsp;<=&nbsp;1:<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;n<br>&nbsp;&nbsp;else:<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;fibonacci(n-1)&nbsp;+&nbsp;fibonacci(n-2)<br><br>n&nbsp;=&nbsp;int(input(""Enter&nbsp;a&nbsp;positive&nbsp;integer:&nbsp;""))<br>print(""Fibonacci&nbsp;sequence&nbsp;up&nbsp;to&nbsp;{}:&nbsp;"".format(n))<br>for&nbsp;i&nbsp;in&nbsp;range(n):<br>&nbsp;&nbsp;print(fibonacci(i))<br>def&nbsp;zero_pad(s):<br>&nbsp;&nbsp;new_s&nbsp;=&nbsp;s.rjust(10,&nbsp;'0')&nbsp;#&nbsp;'0'&nbsp;is&nbsp;the&nbsp;character&nbsp;to&nbsp;be&nbsp;padded&nbsp;with,&nbsp;rjust()&nbsp;is&nbsp;the&nbsp;function&nbsp;for&nbsp;right&nbsp;justification<br>&nbsp;&nbsp;return&nbsp;new_s<br>string&nbsp;=&nbsp;input(""Enter&nbsp;a&nbsp;string&nbsp;to&nbsp;be&nbsp;padded&nbsp;with&nbsp;zeros:&nbsp;"")<br>print(zero_pad(string))<br>","[\ndef fibonacci(n):\n if n < 0:\n return ""Invalid input!!! Input should be a positive integer.""\n elif n <= 1:\n return n\n else:\n return fibonacci(n-1) + fibonacci(n-2)\n\nn = int(input(""Enter a positive integer: ""))\nprint(""Fibonacci sequence up to {}: "".format(n))\nfor i in range(n):\n print(fibonacci(i))\n, \ndef zero_pad(s):\n new_s = s.rjust(10, '0') # '0' is the character to be padded with, rjust() is the function for right justification\n return new_s\nstring = input(""Enter a string to be padded with zeros: "")\nprint(zero_pad(string))\n]","Code for fibonacci sequence:\n```python\ndef fibonacci(n):\n if n < 0:\n return ""Invalid input!!! Input should be a positive integer.""\n elif n <= 1:\n return n\n else:\n return fibonacci(n-1) + fibonacci(n-2)\n\nn = int(input(""Enter a positive integer: ""))\nprint(""Fibonacci sequence up to {}: "".format(n))\nfor i in range(n):\n print(fibonacci(i))\n```\nThis code takes a positive integer, 'n' as input and prints the fibonacci sequence up to 'n'.\n\nCode for zero padding a string to 10 chars:\n```python\ndef zero_pad(s):\n new_s = s.rjust(10, '0') # '0' is the character to be padded with, rjust() is the function for right justification\n return new_s\nstring = input(""Enter a string to be padded with zeros: "")\nprint(zero_pad(string))\n```\nThis code takes an input string, pads zeros to left of the string and makes it total 10 characters long. Here, rjust function of string type is used to pad zeros! This will help in forming names and salutations for certain transcript files, for example:-\n>Automatically formatting the output of strings representing numbers into preset width columns, with an appropriate number of leading zeros... Ensure consistency across a longer range of error / dev results."


In [9]:
# A pandas style applies custom formatting to a DataFrame to make it easier to read and
# understand. It does not offer the full range of functionality that a DataFrame provides.
response_parser.styled_log

,id,created,model,usage,all_code
0,chatcmpl-756VFQYJPiS3pWVPJZ6oVdGWjlolU,1681450993.000000,gpt-3.5-turbo-0301,66.000000,"def fibonacci(n): if n < 0: return ""Invalid input!!! Input should be a positive integer."" elif n <= 1: return n else: return fibonacci(n-1) + fibonacci(n-2)n = int(input(""Enter a positive integer: ""))print(""Fibonacci sequence up to {}: "".format(n))for i in range(n): print(fibonacci(i))def zero_pad(s): new_s = s.rjust(10, '0') # '0' is the character to be padded with, rjust() is the function for right justification return new_sstring = input(""Enter a string to be padded with zeros: "")print(zero_pad(string))"


In [10]:
# this is a PromptLogs object again. We'll add the response_parser to the logs
logs.append(messages, response_parser)
logs.styled_log

[{'role': 'system', 'content': 'You are a software expert.Use. "```python" style for code blocks.Include the name for a file where I would store individual code for functions or classes.'}, {'role': 'user', 'content': 'Write python code for fibonacci sequence. also write a function to zero pad a string to 10 chars.'}]
df: Empty DataFrame
Columns: []
Index: []


,id,created,model,usage,prompts,all_code,content
sid,,,,,,,
a1,chatcmpl-756VFQYJPiS3pWVPJZ6oVdGWjlolU,1681450993.000000,gpt-3.5-turbo-0301,66.000000,"[{'role': 'system', 'content': 'You are a software expert.Use. ""```python"" style for code blocks.Include the name for a file where I would store individual code for functions or classes.'}, {'role': 'user', 'content': 'Write python code for fibonacci sequence. also write a function to zero pad a string to 10 chars.'}]","def fibonacci(n): if n < 0: return ""Invalid input!!! Input should be a positive integer."" elif n <= 1: return n else: return fibonacci(n-1) + fibonacci(n-2)n = int(input(""Enter a positive integer: ""))print(""Fibonacci sequence up to {}: "".format(n))for i in range(n): print(fibonacci(i))def zero_pad(s): new_s = s.rjust(10, '0') # '0' is the character to be padded with, rjust() is the function for right justification return new_sstring = input(""Enter a string to be padded with zeros: "")print(zero_pad(string))","Code for fibonacci sequence: ```python def fibonacci(n): if n < 0: return ""Invalid input!!! Input should be a positive integer."" elif n <= 1: return n else: return fibonacci(n-1) + fibonacci(n-2) n = int(input(""Enter a positive integer: "")) print(""Fibonacci sequence up to {}: "".format(n)) for i in range(n): print(fibonacci(i)) ``` This code takes a positive integer, 'n' as input and prints the fibonacci sequence up to 'n'. Code for zero padding a string to 10 chars: ```python def zero_pad(s): new_s = s.rjust(10, '0') # '0' is the character to be padded with, rjust() is the function for right justification return new_s string = input(""Enter a string to be padded with zeros: "") print(zero_pad(string)) ``` This code takes an input string, pads zeros to left of the string and makes it total 10 characters long. Here, rjust function of string type is used to pad zeros! This will help in forming names and salutations for certain transcript files, for example:- >Automatically formatting the output of strings representing numbers into preset width columns, with an appropriate number of leading zeros... Ensure consistency across a longer range of error / dev results."


## And we have another exchange

In [11]:
message_list = f"""
system: {sys_prompt_string}
user: Write a python function that returns numbers 1-10 in a list. 
"""

message_parser = MessageParser(message_list)
messages = message_parser.parse()
response = openai.ChatCompletion.create(model='gpt-3.5-turbo', temperature=1.3, max_tokens=300, messages = messages)
response_parser = ResponseParser(response=response)

logs.append(messages, response_parser)

[{'role': 'system', 'content': 'You are a software expert.Use. "```python" style for code blocks.Include the name for a file where I would store individual code for functions or classes.'}, {'role': 'user', 'content': 'Write a python function that returns numbers 1-10 in a list.'}]
df:                                        id       created               model  usage                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [12]:
logs.df

,id,created,model,usage,all_code,code_blocks,content,sid,prompts
0,chatcmpl-756VFQYJPiS3pWVPJZ6oVdGWjlolU,1.681451e+09,gpt-3.5-turbo-0301,66.0,"def&nbsp;fibonacci(n):<br>&nbsp;&nbsp;if&nbsp;n&nbsp;<&nbsp;0:<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;""Invalid&nbsp;input!!!&nbsp;Input&nbsp;should&nbsp;be&nbsp;a&nbsp;positive&nbsp;integer.""<br>&nbsp;&nbsp;elif&nbsp;n&nbsp;<=&nbsp;1:<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;n<br>&nbsp;&nbsp;else:<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;fibonacci(n-1)&nbsp;+&nbsp;fibonacci(n-2)<br><br>n&nbsp;=&nbsp;int(input(""Enter&nbsp;a&nbsp;positive&nbsp;integer:&nbsp;""))<br>print(""Fibonacci&nbsp;sequence&nbsp;up&nbsp;to&nbsp;{}:&nbsp;"".format(n))<br>for&nbsp;i&nbsp;in&nbsp;range(n):<br>&nbsp;&nbsp;print(fibonacci(i))<br>def&nbsp;zero_pad(s):<br>&nbsp;&nbsp;new_s&nbsp;=&nbsp;s.rjust(10,&nbsp;'0')&nbsp;#&nbsp;'0'&nbsp;is&nbsp;the&nbsp;character&nbsp;to&nbsp;be&nbsp;padded&nbsp;with,&nbsp;rjust()&nbsp;is&nbsp;the&nbsp;function&nbsp;for&nbsp;right&nbsp;justification<br>&nbsp;&nbsp;return&nbsp;new_s<br>string&nbsp;=&nbsp;input(""Enter&nbsp;a&nbsp;string&nbsp;to&nbsp;be&nbsp;padded&nbsp;with&nbsp;zeros:&nbsp;"")<br>print(zero_pad(string))<br>","[\ndef fibonacci(n):\n if n < 0:\n return ""Invalid input!!! Input should be a positive integer.""\n elif n <= 1:\n return n\n else:\n return fibonacci(n-1) + fibonacci(n-2)\n\nn = int(input(""Enter a positive integer: ""))\nprint(""Fibonacci sequence up to {}: "".format(n))\nfor i in range(n):\n print(fibonacci(i))\n, \ndef zero_pad(s):\n new_s = s.rjust(10, '0') # '0' is the character to be padded with, rjust() is the function for right justification\n return new_s\nstring = input(""Enter a string to be padded with zeros: "")\nprint(zero_pad(string))\n]","Code for fibonacci sequence:\n```python\ndef fibonacci(n):\n if n < 0:\n return ""Invalid input!!! Input should be a positive integer.""\n elif n <= 1:\n return n\n else:\n return fibonacci(n-1) + fibonacci(n-2)\n\nn = int(input(""Enter a positive integer: ""))\nprint(""Fibonacci sequence up to {}: "".format(n))\nfor i in range(n):\n print(fibonacci(i))\n```\nThis code takes a positive integer, 'n' as input and prints the fibonacci sequence up to 'n'.\n\nCode for zero padding a string to 10 chars:\n```python\ndef zero_pad(s):\n new_s = s.rjust(10, '0') # '0' is the character to be padded with, rjust() is the function for right justification\n return new_s\nstring = input(""Enter a string to be padded with zeros: "")\nprint(zero_pad(string))\n```\nThis code takes an input string, pads zeros to left of the string and makes it total 10 characters long. Here, rjust function of string type is used to pad zeros! This will help in forming names and salutations for certain transcript files, for example:-\n>Automatically formatting the output of strings representing numbers into preset width columns, with an appropriate number of leading zeros... Ensure consistency across a longer range of error / dev results.",a1,"[{'role': 'system', 'content': 'You are a software expert.Use. ""```python"" style for code blocks.Include the name for a file where I would store individual code for functions or classes.'}, {'role': 'user', 'content': 'Write python code for fibonacci sequence. also write a function to zero pad a string to 10 chars.'}]"
0,chatcmpl-756VSmcarOUibsRKxCDMcjUuDvjwk,1.681451e+09,gpt-3.5-turbo-0301,60.0,"def&nbsp;number_list():<br>&nbsp;&nbsp;&nbsp;&nbsp;#Using&nbsp;List&nbsp;Comprehension<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;[i&nbsp;for&nbsp;i&nbsp;in&nbsp;range(1,11)]<br>","[\ndef number_list():\n #Using List Comprehension\n return [i for i in range(1,11)]\n]","Below is the python function that returns a list containing numbers from 1 to 10:\n```python\ndef number_list():\n #Using List Comprehension\n return [i for i in range(1,11)]\n```\nYou can store the above code in the file with a name like `number_functions.py` to keep it clean and organized.",a2,"[{'role': 'system', 'content': 'You are a software expe

### And print the styled logs with the code from two separate prompt exchanges

In [13]:
logs.styled_log

,id,created,model,usage,prompts,all_code,content
sid,,,,,,,
a1,chatcmpl-756VFQYJPiS3pWVPJZ6oVdGWjlolU,1681450993.000000,gpt-3.5-turbo-0301,66.000000,"[{'role': 'system', 'content': 'You are a software expert.Use. ""```python"" style for code blocks.Include the name for a file where I would store individual code for functions or classes.'}, {'role': 'user', 'content': 'Write python code for fibonacci sequence. also write a function to zero pad a string to 10 chars.'}]","def fibonacci(n): if n < 0: return ""Invalid input!!! Input should be a positive integer."" elif n <= 1: return n else: return fibonacci(n-1) + fibonacci(n-2)n = int(input(""Enter a positive integer: ""))print(""Fibonacci sequence up to {}: "".format(n))for i in range(n): print(fibonacci(i))def zero_pad(s): new_s = s.rjust(10, '0') # '0' is the character to be padded with, rjust() is the function for right justification return new_sstring = input(""Enter a string to be padded with zeros: "")print(zero_pad(string))","Code for fibonacci sequence: ```python def fibonacci(n): if n < 0: return ""Invalid input!!! Input should be a positive integer."" elif n <= 1: return n else: return fibonacci(n-1) + fibonacci(n-2) n = int(input(""Enter a positive integer: "")) print(""Fibonacci sequence up to {}: "".format(n)) for i in range(n): print(fibonacci(i)) ``` This code takes a positive integer, 'n' as input and prints the fibonacci sequence up to 'n'. Code for zero padding a string to 10 chars: ```python def zero_pad(s): new_s = s.rjust(10, '0') # '0' is the character to be padded with, rjust() is the function for right justification return new_s string = input(""Enter a string to be padded with zeros: "") print(zero_pad(string)) ``` This code takes an input string, pads zeros to left of the string and makes it total 10 characters long. Here, rjust function of string type is used to pad zeros! This will help in forming names and salutations for certain transcript files, for example:- >Automatically formatting the output of strings representing numbers into preset width columns, with an appropriate number of leading zeros... Ensure consistency across a longer range of error / dev results."
a2,chatcmpl-756VSmcarOUibsRKxCDMcjUuDvjwk,1681451006.000000,gpt-3.5-turbo-0301,60.000000,"[{'role': 'system', 'content': 'You are a software expert.Use. ""```python"" style for code blocks.Include the name for a file where I would store individual code for functions or classes.'}, {'role': 'user', 'content': 'Write a python function that returns numbers 1-10 in a list.'}]","def number_list(): #Using List Comprehension return [i for i in range(1,11)]","Below is the python function that returns a list containing numbers from 1 to 10: ```python def number_list(): #Using List Comprehension return [i for i in range(1,11)] ``` You can store the above code in the file with a name like `number_functions.py` to keep it clean and organized."
